::::
:::{thebe-button}
:::
::::

# Find objects

Find bubble size using two different approaches.


In [ ]:
from devtools import pprint
from geopandas import GeoDataFrame, points_from_xy
from matplotlib.pyplot import subplots
from more_itertools import one, only
from numpy import pi, sqrt
from pandas import DataFrame, IndexSlice, NamedAgg, concat
from seaborn import scatterplot
from shapely import LinearRing, Polygon

from boilercv.data import FRAME, VIDEO
from boilercv.images import scale_bool
from boilercv_docs.nbs import init
from boilercv_pipeline.models.deps import get_slices
from boilercv_pipeline.models.stages import Format, display_options
from boilercv_pipeline.sets import get_contours_df2, get_dataset2
from boilercv_pipeline.stages.common.e230920 import GBC, bounded_ax, const
from boilercv_pipeline.stages.common.e230920.types import DfNbOuts
from boilercv_pipeline.stages.find_objects import Deps
from boilercv_pipeline.stages.find_objects import FindObjects as Params

PARAMS = None
V = const.video_dims
APPROACH = "Approach"
TRACKPY = "Trackpy"
CENTROIDS = "Centroids"
PALETTE = {TRACKPY: "red", CENTROIDS: "blue"}
IMSHOW_FORMAT = Format(scale=1.0)

In [ ]:
if isinstance(PARAMS, str):
    params = Params.model_validate_json(PARAMS)
else:
    params = Params(
        context=(ctx := init()),
        deps=(deps := Deps(context=ctx)),
        compare_with_trackpy=True,
        include_patterns=const.nb_include_patterns,
        slicer_patterns=const.nb_slicer_patterns,
    )
outs = DfNbOuts()
params.format.set_display_options()
contours = get_contours_df2(one(params.contours))
slices = get_slices(one(params.filled_slicers))
frames = slices.get(FRAME, slice(None))
filled = scale_bool(get_dataset2(one(params.filled), slices=slices)[VIDEO])
df = only(params.dfs)


def preview(df: DataFrame) -> DataFrame:
    """Preview dataframes in this notebook."""
    return df.set_index(FRAME).groupby(FRAME).head(3).head(6)


pprint(params)

## Data

Load a video of filled contours and the contour loci and plot a composite of all frames to analyze.


In [ ]:
fig, ax = subplots()
outs.plots.append(fig)
composite_video = filled.max(FRAME).values
with (
    display_options(orig=params.format, new=IMSHOW_FORMAT),
    bounded_ax(composite_video, ax=ax),
):
    ax.imshow(~composite_video, alpha=0.4)
    ax.set_xlabel(V.x.latex)
    ax.set_ylabel(V.y.latex)

## Find size from filled contours using Trackpy

Use Trackpy to find bubble size given the filled contours.


In [ ]:
if params.compare_with_trackpy:
    from trackpy import batch, quiet

    quiet()

    tp_objects = (
        batch(
            frames=filled.values, diameter=params.guess_diameter, characterize=True
        ).assign(
            frame=lambda df: df.frame.replace(dict(enumerate(filled.frame.values)))
        )
    ).loc[:, params.trackpy_cols]
    display(preview(tp_objects))

## Find size from contours

The prior approach throws out contour data, instead operating on filled contours. Instead, try using shapely to find size directly from contour data.

### Prepare to find objects

Prepare a dataframe with columns in a certain order, assign contour data to it, and demote the hiearchical indices to plain columns. Count the number of points in each contour and each frame, keeping only those which have enough points to describe a linear ring. Construct a GeoPandas geometry column and operate on it with Shapely to construct linear rings, returning centroids and the representative polygonal area. Also report the number of points in the loci of each contour per frame.


In [ ]:
from boilercv.data import XPX, YPX

objects = (
    DataFrame(columns=[XPX, YPX])
    .assign(**contours.loc[IndexSlice[frames, :], :])
    .rename(axis="columns", mapper=dict(xpx=V.x.src, ypx=V.y.src))
    .reset_index()
    .assign(count=lambda df: df.groupby([FRAME, "contour"], **GBC).x.transform("count"))
    .query("count > 3")
    .assign(geometry=lambda df: points_from_xy(df.x, df.y))
    .groupby([FRAME, "contour"], **GBC)
    .agg(
        count=NamedAgg(column="count", aggfunc="first"),
        centroid=NamedAgg(
            column="geometry", aggfunc=lambda df: LinearRing(df).centroid
        ),
        area=NamedAgg(column="geometry", aggfunc=lambda df: Polygon(df).area),
    )
    .assign(
        diameter_px=lambda df: sqrt(4 * df["area"] / pi),
        radius_of_gyration_px=lambda df: df["diameter_px"] / 4,
        size=lambda df: df["radius_of_gyration_px"],
    )
)
preview(objects)

Split the centroid point objects into separate named columns that conform to the Trackpy convention. Report the centroids in each frame.


In [ ]:
outs = DfNbOuts(
    df=(
        GeoDataFrame(objects)
        .assign(x=lambda df: df.centroid.x, y=lambda df: df.centroid.y)
        .loc[:, params.cols]
    )
)
preview(outs.df)

## Compare approaches

Compare Trackpy objects with contour objects. Here the guess radius for Trackpy object finding and contour perimeter filtering are matched to produce the same number of objects from each algorithm. Trackpy features more intelligent filtering, but takes much longer. Trackpy's approach for finding local maxima in grayscale images is applied even to binarized images, exhaustively searching for high points in the binary image, adding to execution time.

The percent difference between the approaches is relatively low for this subset, suggesting the contour centroid approach is reasonable.

A warm color palette is used to plot Trackpy objects, and a cool color palette is used to plot contour centroids.


In [ ]:
with display_options(orig=params.format, new=IMSHOW_FORMAT):
    scatterplot(
        ax=ax,
        alpha=0.5,
        x=V.x.src,
        y=V.y.src,
        hue=APPROACH,
        palette=PALETTE,
        data=concat([
            (
                tp_objects[[V.y.src, V.x.src]].assign(**{APPROACH: TRACKPY})
                if params.compare_with_trackpy
                else DataFrame()
            ),
            outs.df[[V.y.src, V.x.src]].assign(**{APPROACH: CENTROIDS}),
        ]),
        legend=params.compare_with_trackpy,
    )
    if params.compare_with_trackpy:
        params.format.move_legend(ax, ncol=1)
fig